In [ ]:
import os
from openai import OpenAI
import anthropic
import ipywidgets as widgets
from IPython.display import display, Markdown

# Make sure you source your .env file beforehand in the terminal!
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

client = OpenAI(api_key=openai_api_key)
anthropic_client = anthropic.Anthropic(api_key=anthropic_api_key)

In [ ]:
from datetime import datetime

def save_conversation_to_file(char_a, char_b, topic, messages):
    timestamp = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    filename = f"history/{char_a} vs {char_b} {timestamp}.md"

    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"{char_a} vs {char_b}\n")
        f.write(f"Debate Topic: {topic}\n\n")
        for speaker, text in messages:
            f.write(f"{speaker}:\n{text}\n\n")
    
    print(f"Saved debate as: {filename}")

In [ ]:
import random

characters = [
    "Dwight Schrute", "Sheldon Cooper", "Tony Soprano", "Hank Hill", "George Costanza", 
    "Dr. Evil", "The Dude", "Michael Scott", "Mike Brady", "Wayne Campbell"
]

random.shuffle(characters) 

fighter_a = widgets.Dropdown(options=characters, description="Fighter A:")
fighter_b = widgets.Dropdown(options=characters, description="Fighter B:")
fight_button = widgets.Button(description="FIGHT!", button_style='danger')

display(fighter_a, fighter_b, fight_button)

def generate_debate_topic(character_name):
    prompt = (
        f"You are {character_name}, known for your unique personality and comedic style. "
        "Suggest a funny debate topic. "
        "Keep it short and specific. Return only the topic, not a full explanation."
    )

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

def chatbot_battle(char_a, char_b, output, turns=3):
    messages = []
    topic = generate_debate_topic(char_a)

    with output:
        output.clear_output()
        display(Markdown(f"### {char_a} vs. {char_b}"))
        display(Markdown(f"#### Debate Topic: *{topic}*"))

        chatgpt_line = f"{char_a}: {topic}"
        messages.append((char_a, chatgpt_line))
        display(Markdown(f"**{char_a}:** {topic}"))

        for _ in range(turns):
            try:
                claude_resp = anthropic_client.messages.create(
                    model="claude-3-7-sonnet-latest", # or: claude-3-opus-latest
                    max_tokens=400,
                    temperature=0.7,
                    messages=[{
                        "role": "user",
                        "content": f"Respond in the voice of {char_b} to this statement by {char_a}:\n\"{chatgpt_line}\""
                    }]
                ).content[0].text.strip()
            except Exception as e:
                claude_resp = f"[Claude failed to respond: {e}]"
            messages.append((char_b, claude_resp))
            display(Markdown(f"**{char_b}:** {claude_resp}"))

            try:
                chatgpt_resp = client.chat.completions.create(
                    model="gpt-4",
                    temperature=0.7,
                    max_tokens=400,
                    messages=[{
                        "role": "user",
                        "content": f"Respond in the voice of {char_a} to this statement by {char_b}:\n\"{claude_resp}\""
                    }]
                ).choices[0].message.content.strip()
            except Exception as e:
                chatgpt_resp = f"[ChatGPT failed to respond: {e}]"
            messages.append((char_a, chatgpt_resp))
            display(Markdown(f"**{char_a}:** {chatgpt_resp}"))

            chatgpt_line = chatgpt_resp

    return topic, messages

# Fight Display Logic
output = widgets.Output()
display(output)

def run_fight(_):
    output.clear_output()
    with output:
        display(Markdown("⏳ **Please wait for Model Kombat to begin...**"))
    try:
        char_a = fighter_a.value
        char_b = fighter_b.value
        if char_a == char_b:
            with output:
                output.clear_output()
                display(Markdown("❌ Fighters must be different!"))
            return
        topic, messages = chatbot_battle(char_a, char_b, output)
        save_conversation_to_file(char_a, char_b, topic, messages)
    except Exception as e:
        import traceback
        with output:
            output.clear_output()
            display(Markdown("❌ **Error occurred:**"))
            display(Markdown(f"```{traceback.format_exc()}```"))


fight_button.on_click(run_fight)